In [6]:
#Import all the packages
#import pandas as pd
import numpy as np
from collections import OrderedDict
from neo4j import GraphDatabase
import json
import itertools
import os

In [7]:
#Read the requirement chain from the file
path1 = os.curdir
path2 = "\\RequirementChain.txt"
file_path  = path1 + path2
#Read the contents of the file
with open(file_path, mode="r") as d_f: 
    file_content = d_f.readline()
    RequirementChain = json.loads(file_content)

In [13]:
RequirementChain[11]

{'block_number': 33,
 'block_hash': '0x0f36560fc3b0a5078314301b40f603b60c5f9877a11a1840528a3329e1d1420d',
 'previous_hash': '0xc42976821622914087db51876102cf6d3b61f4785d46212744ba8f7457dac50a',
 'transactions': [{'transaction_hash': '0x3b2070178b1c7acb40701e9fd71424757743886f5f46be0d926bc8586925b607',
   'transaction_data': {'name': 'recordReqData',
    'params': [{'name': 'tx_creator1',
      'value': '0x9e8090b3c5a0dec86c0be4024bbddf9d6745d47a',
      'type': 'address'},
     {'name': 'artifact_trace1',
      'value': ['0xcde8c7a20c9da8dca2114273f6808ad2320c603c3bfe3f457110d499f3426b46',
       'DEPENDS_ON'],
      'type': 'string[]'},
     {'name': 'req_trace1',
      'value': ['The system shall be equipped with GPS system',
       'efabf5ba12e25a2890ef44108455bb97d5ed94403d032094958d3d31d9dcbf24',
       'Develop an anti-lock braking system (ABS) for the bicycles that enables superior control while riding them and prevents accidents',
       '14ed789bb6ef4cf8d542a788fce171616171561

In [9]:
#Function to construct neo4j graph commands
def neo4j_commands(RequirementChain):
    transaction_execution_commands = []
    pos = 0
    total_objects = []
    creators_address = []
    neo4j_create_statement = "CREATE "
    #print(RequirementChain["0"].keys())
    for i in RequirementChain:
        #Creating blocks and connections between them
        #print(i)
        block_number = i['block_number']
        neo4j_create_statement = neo4j_create_statement + "(block" +  str(block_number) 
        neo4j_create_statement = neo4j_create_statement + ":block{name: \"block" +  str(block_number) + "\","
        for j, k in enumerate(i):
            #print(k)
            #print(k)
            if k == "previous_hash" or k == "block_hash":
                neo4j_create_statement = neo4j_create_statement + str(k) +': \"' + str(i[k]) + "\","
        neo4j_create_statement = neo4j_create_statement[:-1]
        neo4j_create_statement = neo4j_create_statement + "}),"
        
        #defining relations between blocks
        if pos >0:
            #pre_block_number = i['block_number']-1
            neo4j_create_statement = neo4j_create_statement + "(block" + str(pre_block_number ) 
            neo4j_create_statement = neo4j_create_statement + ") -[r" + str(pre_block_number ) + ":CONNECTS]-> " 
            neo4j_create_statement = neo4j_create_statement + "(block" + str(block_number) +")," 

        pre_block_number = i['block_number']
        #Creating artifacts and connecting them to blocks
        #x_2 = 0
        #print(i["transactions"])
        tx_creators = []
        sk_names = []
        for x_1 in i["transactions"]:
            #print(x_1)
            #neo4j_create_statement = neo4j_create_statement + "(artifact" +  str(pos+x_2) +":artifact{"
            neo4j_create_statement = neo4j_create_statement + "(art" + str(x_1["transaction_hash"]) +":artifact{name: \" "
            neo4j_create_statement = neo4j_create_statement + str(x_1["transaction_data"]["params"][4]["value"][0]) + "\","
            neo4j_create_statement = neo4j_create_statement + "artifact_hash" 
            neo4j_create_statement = neo4j_create_statement + ': \"'
            neo4j_create_statement = neo4j_create_statement + str(x_1["transaction_hash"]) + "\","
            neo4j_create_statement = neo4j_create_statement + "stake_holders" 
            neo4j_create_statement = neo4j_create_statement + ': \"'
            neo4j_create_statement = neo4j_create_statement + str(x_1["transaction_data"]["params"][3]["value"]) + "\","
            neo4j_create_statement = neo4j_create_statement + "artifact_creator_address" 
            neo4j_create_statement = neo4j_create_statement + ': \"'
            neo4j_create_statement = neo4j_create_statement + str(x_1["transaction_data"]["params"][0]["value"]) + "\","
            if (str(x_1["transaction_data"]["params"][0]["value"]) not in creators_address):
                creators_address.append(str(x_1["transaction_data"]["params"][0]["value"]))
                tx_creators.append(str(x_1["transaction_data"]["params"][0]["value"]))
                sk_names.append(str(x_1["transaction_data"]["params"][3]["value"]))
            neo4j_create_statement = neo4j_create_statement + "Objects" 
            neo4j_create_statement = neo4j_create_statement + ': \"'
            for x_2 in range(0,len(x_1["transaction_data"]["params"][2]["value"]), 5):
                neo4j_create_statement = neo4j_create_statement + str(x_1["transaction_data"]["params"][2]["value"][x_2].replace('"',"'"))
                neo4j_create_statement = neo4j_create_statement + ","
            neo4j_create_statement = neo4j_create_statement + "\","
            neo4j_create_statement = neo4j_create_statement[:-1]
            neo4j_create_statement = neo4j_create_statement + "}),"

        for x_1 in i["transactions"]:
            neo4j_create_statement = neo4j_create_statement + "(block" + str(block_number) 
            neo4j_create_statement = neo4j_create_statement + ") -[a" + str(pos) + "_" + str(x_2) + ":DOCUMENTS]-> " 
            neo4j_create_statement = neo4j_create_statement + "(art" + str(x_1["transaction_hash"]) +"),"
            x_2 = x_2 + 1

        #Creating artifacts and connecting them to blocks
        x_2 = 0
        #print(i["transactions"])
        if (tx_creators):
            for x_1 in tx_creators:
                #print(x_1)
                #neo4j_create_statement = neo4j_create_statement + "(artifact" +  str(pos+x_2) +":artifact{"
                neo4j_create_statement = neo4j_create_statement + "(sk" + str(x_1) + ":stakeholders{name:"
                neo4j_create_statement = neo4j_create_statement + ' \"'+  str(x_1) + '\"' 
                neo4j_create_statement = neo4j_create_statement + ",stake_holders:" 
                neo4j_create_statement = neo4j_create_statement + ' \"' 
                neo4j_create_statement = neo4j_create_statement + str(sk_names[x_2]) + "\"}),"
                #neo4j_create_statement = neo4j_create_statement[:-1]
                #neo4j_create_statement = neo4j_create_statement + "\"}),"
                x_2 = x_2 + 1
        
        x_2 = 0
        for x_1 in i["transactions"]:
            #defining relations between block and creators
            neo4j_create_statement = neo4j_create_statement + "(sk" + str(x_1["transaction_data"]["params"][0]["value"])
            #neo4j_create_statement = neo4j_create_statement + "(block" + str(pos) 
            neo4j_create_statement = neo4j_create_statement + ") -[skb" + str(pos) + '_' + str(x_2) + ":CREATES]-> " 
            neo4j_create_statement = neo4j_create_statement + "(art" + str(x_1["transaction_hash"])  +"),"
            x_2 = x_2 + 1
        
        #defining relations between artifact and artifact
        jpos = 1
        for x_1 in i["transactions"]:
            if pos>0:
                art_trace = x_1["transaction_data"]["params"][1]["value"]
                link_temp = str(art_trace[1]).split(" ")
                link_true = ""
                for lk in link_temp:
                    link_true = link_true + str(lk) + "_"
                link_true2 = link_true[:-1]
                if str(link_true2) == "EVOLVED_TO" or str(link_true2) == "SATISFIES":
                    neo4j_create_statement = neo4j_create_statement + "(art" + str(art_trace[0])
                    neo4j_create_statement = neo4j_create_statement + ") -[aa" + str(pos) + '_' +str(jpos)  
                    #neo4j_create_statement = neo4j_create_statement + ":" + str(j[1]) + "]-> " 
                    neo4j_create_statement = neo4j_create_statement + ":" + link_true2 + "]-> "
                    neo4j_create_statement = neo4j_create_statement +  "(art" + str(x_1["transaction_hash"]) +"),"
                else: 
                    neo4j_create_statement = neo4j_create_statement + "(art" + str(x_1["transaction_hash"])
                    neo4j_create_statement = neo4j_create_statement + ") -[aa" + str(pos) + '_' + str(jpos) 
                    #neo4j_create_statement = neo4j_create_statement + ":" + str(j[1]) + "]-> " 
                    neo4j_create_statement = neo4j_create_statement + ":" + link_true2 + "]-> "
                    neo4j_create_statement = neo4j_create_statement +  "(art" + str(art_trace[0]) +"),"
                jpos = jpos+1
                

        
        #Creating objects and connecting them to blocks
        for x_1 in i["transactions"]:
            Object=[]
            temp_object = x_1["transaction_data"]["params"][2]["value"]
            for x_3 in range(0, len(temp_object), 5):
                Object.append([temp_object[x_3+1], temp_object[x_3+0], x_1["transaction_hash"], x_1["transaction_data"]["params"][0]["value"]])
                #total_objects.append(i[k][p][1])

            Object.sort
            Object2 = list(k for k,_ in itertools.groupby(Object))
            temp_objects = []
            for oi in range(len(Object2)):
                if Object2[oi][0] not in total_objects and Object2[oi][0] not in temp_objects:
                    neo4j_create_statement = neo4j_create_statement + "(obj" + str(Object2[oi][0]) + ":Object{name:"
                    neo4j_create_statement = neo4j_create_statement +' \"' + str(Object2[oi][1].replace('"',"'")) 
                    neo4j_create_statement = neo4j_create_statement+ '\"' + ",public_key:" 
                    neo4j_create_statement = neo4j_create_statement+ ' \"' + str(Object2[oi][3].replace('"',"'")) + "\"}),"
                    temp_objects.append(Object2[oi][0])
                neo4j_create_statement = neo4j_create_statement + "(art" + str(Object2[oi][2]) 
                neo4j_create_statement = neo4j_create_statement + ") -[ar" + str(pos) + "_" + str(oi) 
                neo4j_create_statement = neo4j_create_statement + ":CONTAINS_OBJECT]-> " 
                neo4j_create_statement = neo4j_create_statement + "(obj" + str(Object2[oi][0]) +"),"

            for x_3 in range(0, len(temp_object), 5):
                total_objects.append(temp_object[x_3+1])

        
        #defining relations between object and object
        jpos = 1
        if pos > 0:
            for x_1 in i["transactions"]:
                #print(x_1["transaction_hash"])
                temp_object = x_1["transaction_data"]["params"][2]["value"]
                for x_3 in range(0, len(temp_object), 5):
                    obj_trace = [temp_object[x_3+1], temp_object[x_3+3], temp_object[x_3+4]]
                    #print(obj_trace)
                    link_temp = str(obj_trace[2]).split(" ")
                    if (len(link_temp)):
                        link_temp=str(obj_trace[2]).split("-")
                    link_true = ""
                    for lk in link_temp:
                        link_true = link_true + str(lk) + "_"
                    link_true2 = link_true[:-1]
                    if str(link_true2) == "EVOLVED_TO" or str(link_true2) == "SATISFIES":
                        neo4j_create_statement = neo4j_create_statement + "(obj" + str(obj_trace[1])
                        neo4j_create_statement = neo4j_create_statement + ") -[obj" + str(pos) + "_" + str(jpos)  
                        #neo4j_create_statement = neo4j_create_statement + ":" + str(j[1]) + "]-> " 
                        neo4j_create_statement = neo4j_create_statement + ":" + str(link_true2) + "]-> "
                        neo4j_create_statement = neo4j_create_statement +  "(obj" + str(obj_trace[0]) +"),"
                    else: 
                        neo4j_create_statement = neo4j_create_statement + "(obj" + str(obj_trace[0])
                        neo4j_create_statement = neo4j_create_statement + ") -[obj" + str(pos) + "_"+ str(jpos) 
                        #neo4j_create_statement = neo4j_create_statement + ":" + str(j[1]) + "]-> " 
                        neo4j_create_statement = neo4j_create_statement + ":" + str(link_true2) + "]-> "
                        neo4j_create_statement = neo4j_create_statement +  "(obj" + str(obj_trace[1]) +"),"
                    jpos = jpos+1
        

        pos = pos + 1
    transaction_execution_commands.append(neo4j_create_statement[:-1])
        #neo4j_create_statement = []
    return(transaction_execution_commands)
#Function to execute the neo4j graph commands
def execute_requirementblock(transaction_execution_commands):
    data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "123456"))
    #data_base_connection = GraphDatabase.driver(uri = "http://127.0.0.10:5000/", auth=("neo4j", "password"))
    session = data_base_connection.session()
    for i in transaction_execution_commands:
        session.run(i)
        #print("i am good here at pos {}", i)

In [11]:
#tem  = [RequirementChain[0],RequirementChain[1]]
transaction_execution_commands = neo4j_commands(RequirementChain)
#print(transaction_execution_commands)
execute_requirementblock(transaction_execution_commands)